# MGTA 415 - Session 0 Demo: A simple example of paper trading with Alpaca API

This notebook demonstrates how to use the Alpaca API to pull real-time news, analyze sentiment, and make trading decisions based on the sentiment analysis.

## Step 1: Import Libraries

First, we'll import the necessary libraries and import the secrets. 
This assumes the secrets are already added in .env file.  Everytime you update the .env file you'll need to rebuild the container.

In [1]:
import os
import alpaca_trade_api as tradeapi
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import requests

In [2]:
ALPACA_URL = 'https://paper-api.alpaca.markets'
SYMBOLS = ["AAPL"]

In [3]:
# Alpaca API credentials
ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

# Initialize Alpaca API
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_URL)

# Initialize Sentiment Analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

## Step 2: Retrieve News from Previous Day

We'll retrieve news articles from the previous day using Alpaca's API.


In [4]:
# Function to retrieve news from the previous day for specific symbols
def get_previous_day_news(symbols):
    today = datetime.utcnow()
    yesterday = today - timedelta(days=1)
    url = "https://data.alpaca.markets/v1beta1/news"
    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }
    params = {
        "symbols": ",".join(symbols),
        "start": yesterday.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "end": today.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "limit": 50
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()['news']

# Retrieve news for specific symbols
news = get_previous_day_news(SYMBOLS)


## Step 3: Analyze Sentiment and Make Trading Decisions

We'll analyze the sentiment of the news articles and make trading decisions based on the sentiment scores.


In [6]:
# Function to analyze sentiment and make trading decisions
def analyze_and_trade(news):
    for article in news:
        headline = article['headline']
        summary = article.get('summary', '')
        content = article.get('content', '')
        combined_text = f"{headline} {summary} {content}"
        
        sentiment = sentiment_analyzer.polarity_scores(combined_text)
        sentiment_score = sentiment['compound']
        
        print(f"Headline: {headline}")
        print(f"Sentiment Score: {sentiment_score}")
        
        symbols = article['symbols']
        for symbol in symbols:
            if sentiment_score > 0.05:
                make_trade_decision(symbol, 'buy')
            elif sentiment_score < -0.05:
                make_trade_decision(symbol, 'sell')

In [7]:
# Function to make trading decisions
def make_trade_decision(symbol, action):
    qty = 10  # Example quantity
    
    try:
        if action == 'buy':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            print(f"Buying {qty} shares of {symbol}")
        elif action == 'sell':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            print(f"Selling {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")


# Analyze and trade based on the news
analyze_and_trade(news)

Headline: Inflation Drop Fuels Rate Cut Speculation, Investors Shift To Sector Laggards, Russell 2000 Surges Over 3.6%: This Week In The Markets
Sentiment Score: -0.6369
Selling 10 shares of AAPL
Selling 10 shares of GOOGL
Selling 10 shares of IWM
Selling 10 shares of META
Selling 10 shares of MSFT
Selling 10 shares of PFE
Selling 10 shares of RSP
Selling 10 shares of SPY
Selling 10 shares of XLRE
Headline: AI Super Bulls Pay Attention To The Biggest Divergence Since 2001, TSLA Call Buyers Crushed
Sentiment Score: 0.6249
Error placing order: potential wash trade detected. use complex orders
Buying 10 shares of AMZN
Error placing order: order cost basis must not exceed the notional limit of $200000
Buying 10 shares of C
Buying 10 shares of FAST
Buying 10 shares of GLD
Buying 10 shares of GOOG
Error placing order: potential wash trade detected. use complex orders
Buying 10 shares of JPM
Buying 10 shares of KBH
Error placing order: potential wash trade detected. use complex orders
Error p